In [4]:
import numpy as np
import pandas as pd

# 定义训练数据
data = [['Sunny', 'Hot', 'High', 'Weak', 'No'], ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], ['Sunny', 'Mild', 'High', 'Weak', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
        ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'], ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Strong', 'No']
        ]

# 创建数据框
Data = pd.DataFrame(data, columns=['Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis'])

# 打印数据
print(Data)

     Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No


In [5]:
# 提取特征和标签
cols = Data.shape[1]
X_data = Data.iloc[:, :cols - 1]
Y_data = Data.iloc[:, cols - 1:]
featureNames = X_data.columns

# 对标签进行唯一化处理
y = Y_data.values
y_unique = np.unique(y)
x = X_data.values

In [6]:
# 创建测试数据
test_data = [['Sunny', 'Cool', 'High', 'Strong']]
test_data = pd.DataFrame(test_data, columns=['Outlook', 'Temperature', 'Humidity', 'Wind'])
print(test_data)

  Outlook Temperature Humidity    Wind
0   Sunny        Cool     High  Strong


In [7]:
# 定义朴素贝叶斯函数
def Naive_Bayes(X_data, Y_data, featureNames):
    y = Y_data.values
    x = X_data.values
    y_unique = np.unique(y)
    prior_prob = np.zeros(len(y_unique))
    for i in range(len(y_unique)):
        prior_prob[i] = np.sum(y == y_unique[i]) / len(y)

    condition_prob = {}

    for feat in featureNames:
        x_unique = list(set(X_data[feat]))
        x_condition_prob = np.zeros((len(y_unique), len(x_unique)))
        for i in range(len(y_unique)):
            for j in range(len(x_unique)):
                x_condition_prob[i, j] = sum(
                    (X_data[feat] == x_unique[j]) & (Y_data['PlayTennis'] == y_unique[i])) / sum(
                    y == y_unique[i])
        x_condition_prob = pd.DataFrame(x_condition_prob, columns=x_unique, index=y_unique)
        condition_prob[feat] = x_condition_prob

    return prior_prob, condition_prob

In [8]:
# 定义预测函数
def Prediction(test_data, prior, condition_prob, y_unique):
    numclass = prior.shape[0]
    featureNames = test_data.columns
    numsample = test_data.shape[0]
    post_prob = np.zeros((numsample, numclass))
    for k in range(numsample):
        prob_k = np.zeros((numclass,))
        for i in range(numclass):
            pri = prior[i]
            for feat in featureNames:
                feat_val = test_data[feat][k]
                cp = condition_prob[feat]
                cp_val = cp.loc[y_unique[i], feat_val]  # 使用标签值进行索引
                pri *= cp_val
            prob_k[i] = pri
        prob = prob_k / np.sum(prob_k, axis=0)
        post_prob[k, :] = prob

        if (post_prob[0][0] > post_prob[0][1]):
            lable = y_unique[0]
        else:
            lable = y_unique[1]

    return post_prob, lable

In [9]:
# 训练朴素贝叶斯模型并进行预测
prior_prob, condition_prob = Naive_Bayes(X_data, Y_data, featureNames)
post_prob , lable = Prediction(test_data, prior_prob, condition_prob, y_unique)

C:\Users\WXJ\AppData\Local\Temp\ipykernel_12048\1397865830.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_condition_prob[i, j] = sum(
C:\Users\WXJ\AppData\Local\Temp\ipykernel_12048\1397865830.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_condition_prob[i, j] = sum(
C:\Users\WXJ\AppData\Local\Temp\ipykernel_12048\1397865830.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_condition_prob[i, j] = sum(
C:\Users\WXJ\AppData\Local\Temp\ipykernel_1

In [10]:
print(lable, post_prob)  # 输出预测结果和后验概率

No [[0.79541735 0.20458265]]
